In [14]:
import cv2
import numpy as np
import os

In [15]:
def pre_process(path):
    image=cv2.imread(path)
    gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    scale_factor=700/max(image.shape[:2])
    image=cv2.resize(image,(0,0),fx=scale_factor,fy=scale_factor)
    gray=cv2.resize(gray,(0,0),fx=scale_factor,fy=scale_factor)
    blurred=cv2.GaussianBlur(gray,(5,5),0)
    thresh=cv2.adaptiveThreshold(blurred,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,2)
    return (image,thresh,scale_factor)
    
def show(img,time=3,msg="Image"):
    cv2.imshow(msg,img)
    cv2.waitKey(int(time*1000))
    cv2.destroyAllWindows()

def edge(image,thresh,scale_factor):
    contours,_=cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    coin_contours=[]
    for cnt in contours:
        perimeter=cv2.arcLength(cnt,True)
        area=cv2.contourArea(cnt)
        if perimeter:
            circularity=4*np.pi*(area/(perimeter**2))
            if 0.7 < circularity < 1.2 and area > 500*(scale_factor**2):  
                coin_contours.append(cnt)
    cv2.drawContours(image,coin_contours,-1,(0,255,0),2)
    show(image,3,"Edges of coins")
    return coin_contours

    
def contour_segmentation(image,coin_contours):
    segmented_coins=[]
    for cnt in coin_contours:
        (x,y),radius=cv2.minEnclosingCircle(cnt)
        center=(int(x),int(y))
        radius=int(radius)
        mask=np.zeros_like(image,dtype=np.uint8)
        cv2.circle(mask,center,radius,(255,255,255),-1)
        coin_segment=cv2.bitwise_and(image,mask)
        x1,y1,x2,y2=center[0] - radius,center[1] - radius,center[0] + radius,center[1] + radius
        coin_segment=coin_segment[y1:y2,x1:x2]
        segmented_coins.append(coin_segment)
    for i,coin in enumerate(segmented_coins):
        show(coin,2,f'{i+1}th segmented coin')
    return segmented_coins

    
def count_coin(coin_contours,segmented_coins):
    return (len(coin_contours),len(segmented_coins))

In [16]:
def main(path):
    image,thresh,scale_factor=pre_process(path)
    coin_contours=edge(image,thresh,scale_factor)
    segmented_coins=contour_segmentation(image,coin_contours)
    print(count_coin(coin_contours,segmented_coins))

In [17]:
for coin in os.listdir("coins/"):
    main(os.path.join("coins/",coin))